# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: fetch latest code</h2>
            <span style="color:#f71;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use GPT-4o and Claude-3.5-Sonnet, which are the slightly higher priced models. The costs are still low, but if you'd prefer to keep costs ultra low, please make the suggested switches to the models (3 cells down from here).
            </span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')

In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()
qwencoder= OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
phi4= OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

PHI4_MODEL ='phi4'
QWENCODER_MODEL = 'ticlazau/qwen2.5-coder-7b-instruct'
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"


In [4]:
def system_messages(original_lang,lang):
    system_message=f"""
    You are an assistant that reimplements original_code{original_lang} code  in high performance {lang} for a win64 pc. 
    Respond only with {lang} code; use  only  proper code comments. Do not include anything outside strict coding text as response.thing 
    Mark the beginning of code with "<START>" as the first line and with <END> as the last line. Cut everything before and after those markers.
    The {lang} response needs to produce an identical output in the fastest possible time.
    """
    return system_message

In [5]:
def user_prompt_for(original_code,lang):
    user_prompt = f"Rewrite this original_code code in {lang} with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += f"Respond only with strict {lang} code; do not explain your work or add any comment outside code body. "
    user_prompt += f"Pay attention to number types to ensure no int overflows. Remember to include all necessary {lang} packages needed.\n\n"
    user_prompt += original_code
    return user_prompt

In [6]:
def messages_for(original_code,lang,original_lang):
    return [
        {"role": "system", "content": system_messages(lang,original_lang)},
        {"role": "user", "content": user_prompt_for(original_code,lang)}
    ]

In [7]:
# write to a file called optimized.xxx

def write_output(reply,lang):
     
    if len(reply)==0:
        return 'Failed to save file'
        
    # ind1 = reply.find('<START>\n')
    # ind2 = reply.rfind('<END>')
    # code= reply[ind1+1:ind2-1]
    if lang == 'c#':
        filename ='optimised.cs'
    elif lang=='rust':
        filename ='optimised.rs'
    elif lang=='go':
        filename='optimised.go'
    elif lang=='c++':
        filename= 'optimised.cpp'
    try:
        with open(filename, "w") as f:
            f.write(reply)
            return 'File saved'
    except:
        return 'Failed to save file'
    

In [8]:
# def optimize_gpt(python):    
#     stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
#     reply = ""
#     for chunk in stream:
#         fragment = chunk.choices[0].delta.content or ""
#         reply += fragment
#         print(fragment, end='', flush=True)
#     write_output(reply,lang)

In [9]:
# def optimize_claude(python):
#     result = claude.messages.stream(
#         model=CLAUDE_MODEL,
#         max_tokens=2000,
#         system=system_message,
#         messages=[{"role": "user", "content": user_prompt_for(python)}],
#     )
#     reply = ""
#     with result as stream:
#         for text in stream.text_stream:
#             reply += text
#             print(text, end="", flush=True)
#     write_output(reply,lang)

In [10]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

In [ ]:
optimize_gpt(pi)

In [ ]:
optimize_claude(pi)

In [ ]:
optimize_qwen(pi)

In [ ]:
exec(pi)

# Compiling C++ and executing

This next cell contains the command to compile a C++ file on my M1 Mac.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

In the next lab (day4), a student has contributed a full solution that compiles to efficient code on Mac, PC and Linux!

You can wait for this, or you can google (or ask ChatGPT!) for how to do this on your platform, then replace the lines below.
If you're not comfortable with this step, you can skip it for sure - I'll show you exactly how it performs on my Mac.


OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [ ]:
# # Compile C++ and run the executable

# !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
# !./optimized

In [ ]:
#optimize_claude(pi)

In [ ]:
# # Repeat for Claude - again, use the right approach for your platform

# !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
# !./optimized

In [ ]:
original_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
#exec(python_hard)

In [ ]:
#optimize_gpt(python_hard)

In [ ]:
# # Replace this with the right C++ compile + execute command for your platform

# !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
# !./optimized

In [ ]:
#optimize_claude(python_hard)

In [ ]:
#optimize_qwen(python_hard)

In [ ]:
# # Replace this with the right C++ compile + execute command for your platform

# !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
# !./optimized

In [11]:
def stream_gpt(original_code,lang,original_lang):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(original_code,lang,original_lang), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('<START>','').replace('<END>','')

In [12]:
def stream_qwen(original_code,lang,original_lang):    
    stream = qwencoder.chat.completions.create(model=QWENCODER_MODEL, messages=messages_for(original_code,lang,original_lang), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('<START>','').replace('<END>','')

In [13]:
def stream_phi4(original_code,lang,original_lang):    
    stream = phi4.chat.completions.create(model=PHI4_MODEL, messages=messages_for(original_code,lang,original_lang), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('<START>','').replace('<END>','')

In [14]:
def stream_claude(original_code,lang,original_lang):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_messages(lang,original_lang),
        messages=[{"role": "user", "content": user_prompt_for(original_code,lang)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('<START>','').replace('<END>','')

In [15]:
def optimize(original_code, model,lang,original_lang):
    if model=="GPT":
        result = stream_gpt(original_code,lang,original_lang)
    elif model=="Claude":
        result = stream_claude(original_code,lang,original_lang)
    elif model == 'Phi4':
        result = stream_phi4(original_code,lang,original_lang)
    else:
        result = stream_qwen(original_code,lang,original_lang)
    for stream_so_far in result:
        yield stream_so_far        

In [24]:
def upload_code(filename):
    print(filename)
    with open(filename,'r') as file:
        code= file.read()
    print(code)
    return code

In [26]:
with gr.Blocks() as ui:
    lang =''
    original_lang=''
    original_code=''
    
    with gr.Row():
          
        original_code = gr.Textbox(label="Original code:", lines=10,) #value=pi)
        code = gr.Textbox(label=f"{lang} Converted code:", lines=10)
    with gr.Row():
        original_lang=gr.Dropdown(['Python','Php','Java','C++'],label='Select original language')        
        lang = gr.Dropdown(['c++','c#','rust','go'],label='Select new language', value ='c++')
    with gr.Row():
        with gr.Column():
            model = gr.Dropdown(["GPT", "Claude",'Phi4',"Qwen"], label="Select model", value="Qwen")          
            uploadBtn= gr.UploadButton('Import code')
            clearBtn =gr.Button('Clear')
            
        with gr.Column():
            convert = gr.Button("Convert code")
            saveBtn=gr.Button('Save to a file')
            saving=gr.Textbox()

    uploadBtn.upload(upload_code,uploadBtn, outputs=original_code)
    convert.click(optimize, inputs=[original_code, model,lang,original_lang], outputs=[code])
    saveBtn.click(write_output,inputs=[code,lang],outputs=[saving])
    clearBtn.click(lambda: [None,None], outputs=[original_code,code])
    
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


C:\Users\User\AppData\Local\Temp\gradio\36b8a319a23ca52eef7f74f525a573b113b17a9553a5593f0cea4ae4dc44411f\simple.cpp

#include <iostream>

int main() {
    std::cout << "Hello";
    return 0;
}



In [ ]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
# You'll need to change the code in the try block to compile the C++ code for your platform
# I pasted this into Claude's chat UI with a request for it to give me a version for an Intel PC,
# and it responded with something that looks perfect - you can try a similar approach for your platform.

# M1 Mac version to compile and execute optimized C++ code:

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)